### Goal: Generate slices of fmri scans, then use a classfier to train on newly created slices

In [26]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
from pathlib import Path


# SLICE GENERATOR

In [27]:
devices = tf.config.list_physical_devices()
# print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

GPU details:  {'device_name': 'METAL'}


In [28]:
test_img_path = os.path.join('data','fmri','beijing')
savepath = os.path.join('outputs','slice-outputs')
fmri_files = Path("data/fmri/beijing").glob("*.nii.gz")
count = 0

def slice_gen(imagename,x):
    
    slice_data = test_img_data[:,:,20,x]

    plt.imshow(slice_data, cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.savefig(savepath + "/"+ imagename, bbox_inches='tight', pad_inches=0)
    plt.close()

for files in fmri_files:
    imgname = 'output' + str(count) + '.png'
    test_img = nib.load(str(files))
    test_img_data = test_img.get_fdata()
    #for x in range (0, 3):
    slice_gen(imgname,count)
    count+=1

# CLASSIFIER
code from: https://www.kaggle.com/code/stefanie04736/simple-keras-model-with-k-fold-cross-validation


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import keras
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline
keras.backend.set_image_data_format('channels_last')

In [ ]:
def load_data_kfold(k):
    
    train = pd.read_json('../input/train.json')
    train.inc_angle = train.inc_angle.replace('na', 0)
    
    x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
    x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
    x_band3 = x_band1 / x_band2
       
    X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                            , x_band2[:, :, :, np.newaxis]
                            , x_band3[:, :, :, np.newaxis]], axis=-1)
                         
    y_train = np.array(train["is_iceberg"])
    
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(X_train, y_train))
    
    return folds, X_train, y_train

k = 7
folds, X_train, y_train = load_data_kfold(k)

In [ ]:
def get_model():
    
    x = Input((75, 75, 3))
    model = BatchNormalization(axis = 3)(x)
    model = Convolution2D(filters = 32, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 128, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model)
    model = MaxPooling2D()(model)
    
    model = BatchNormalization(axis = 3)(model)
    model = Convolution2D(filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same', activation='relu')(model) 
    model = GlobalAveragePooling2D()(model)
 
    model = Dense(1, activation = 'sigmoid')(model)
    
    model = Model(input = x, output = model)
    
    opt_adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(opt_adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = get_model()
model.summary()

In [ ]:
batch_size=64

gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

In [ ]:
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    return [mcp_save, reduce_lr_loss]

In [ ]:
for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    X_train_cv = X_train[train_idx]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train[val_idx]
    y_valid_cv= y_train[val_idx]
    
    name_weights = "final_model_fold" + str(j) + "_weights.h5"
    callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
    model = get_model()
    model.fit_generator(
                generator,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=15,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv),
                callbacks = callbacks)
    
    print(model.evaluate(X_valid_cv, y_valid_cv))